<a href="https://colab.research.google.com/github/akiabe/coursera-ml-stanford/blob/master/ml_ex2_Logistic_regression_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Build logistic regresion from scratch**


Data set: 2 exams score of university applicant's 

Predict: Estimate probability of admission

In [0]:
# Package imports
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
# Load training data set
data = np.loadtxt(open("ex2data1.txt", "r"), delimiter=",")
X = data[:, 0:2]
y = data[:, 2]
print ("X shape: " + str(X.shape))
print ("y shape: " + str(y.shape))

X shape: (100, 2)
y shape: (100,)


In [18]:
# Add intercept term to X
m = X.shape[0]
X = np.concatenate([np.ones((m, 1)), X], axis=1)
print ("X shape: " + str(X.shape))
print ("y shape: " + str(y.shape))

X shape: (100, 3)
y shape: (100,)


In [0]:
# Sigmoid function
def sigmoid(x):
  s = 1/(1+np.exp(-x))
  return s

In [0]:
# Cost function
def cost_function(theta, X, y):
  m = X.shape[0]
  J = 0
  h = sigmoid(np.dot(X, theta.T))
  J = (1/m)*np.sum(np.dot((-y), np.log(h))-np.dot((1-y), np.log(1-h)))
  return J

In [0]:
# Gradient of cost
def gradient_of_cost(theta, X, y):
  m = X.shape[0]
  J = 0
  grad = np.zeros((theta.shape))
  h = sigmoid(np.dot(X, theta.T))
  grad = (1/m)*(np.dot((h-y), X))
  return grad

In [22]:
# Initialize fitting parameters
initial_theta = np.zeros(X.shape[1])
J = cost_function(initial_theta, X, y)
grad = gradient_of_cost(initial_theta, X, y)

print ("Cost at initial theta: " + str(J))
print ("Gradient at initial theta: " + str(grad))

Cost at initial theta: 0.6931471805599453
Gradient at initial theta: [ -0.1        -12.00921659 -11.26284221]


In [24]:
# Optimize cost function
res = minimize(cost_function,
               initial_theta,
               args=(X,y),
               method=None,
               jac=gradient_of_cost,
               options={'maxiter':400});

cost = res.fun
theta = res.x  # rex.x -- optimized theta values

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: divide by zero encountered in log
  


In [0]:
# Predict
def predict(theta, X):
  m = X.shape[0]
  p = np.zeros(m)
  p = np.round(sigmoid(np.dot(X, theta.T)))
  return p

In [34]:
# Predict admission probability of student with Exam 1 score 45 and Exam 2 score 85
sigmoid(np.dot(np.array([1, 45, 85]), theta.T))

0.7762907240588949

In [35]:
p = predict(theta, X) 
accuracy = np.mean(p==y)*100
print ("% of train accuracy: " + str(accuracy))

% of train accuracy: 89.0
